In [1]:
from dotenv import load_dotenv
import os
from sqlalchemy.engine import URL, Engine
from sqlalchemy import create_engine, Table, MetaData, Column
from sqlalchemy.dialects import postgresql
from sqlalchemy import Table, MetaData, Column, Integer, String, Float

In [2]:
import pandas as pd
# Set option to display all columns
pd.set_option('display.max_columns', None)

In [3]:
faa_data = pd.read_csv("data/AIDS_REPORTS.csv")
faa_data.head()

,AIDS Report Number,Local Event Date,Event City,Event State,Event Airport,Event Type,Aircraft Damage,Flight Phase,Aircraft Make,Aircraft Model,Aircraft Series,Operator,Primary Flight Type,Flight Conduct Code,Flight Plan Filed Code,Aircraft Registration Nbr,Total Fatalities,Total Injuries,Aircraft Engine Make,Aircraft Engine Model,Engine Group Code,Nbr of Engines,PIC Certificate Type,PIC Flight Time Total Hrs,PIC Flight Time Total Make-Model,,.1
0,20240801009689I,01-AUG-24,SAN DIEGO,CA,SAN DIEGO INTL,NaN,NaN,NaN,NaN,NaN,NaN,JAZZ,NaN,FOREIGN AIR CARRIER,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240801010149I,01-AUG-24,MANCHESTER,NH,MANCHESTER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,3200.0,1600.0,2.0,NaN
2,20240801010459I,01-AUG-24,BROOMFIELD,CO,ROCKY MOUNTAIN METROPOLITAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20240801011259I,01-AUG-24,QUANTICO,VA,QUANTICO MCAF /TURNER FIELD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1145.0,NaN,NaN,NaN
4,20240801011459I,01-AUG-24,FORT LAUDERDALE,FL,FORT LAUDERDALE EXECUTIVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,4188.0,232.0,19.0,NaN


In [4]:
faa_data.columns

Index(['AIDS Report Number', 'Local Event Date', 'Event City', 'Event State',
       'Event Airport', 'Event Type', 'Aircraft Damage', 'Flight Phase',
       'Aircraft Make', 'Aircraft Model', 'Aircraft Series', 'Operator',
       'Primary Flight Type', 'Flight Conduct Code', 'Flight Plan Filed Code',
       'Aircraft Registration Nbr', 'Total Fatalities', 'Total Injuries',
       'Aircraft Engine Make', 'Aircraft Engine Model', 'Engine Group Code',
       'Nbr of Engines', 'PIC Certificate Type', 'PIC Flight Time Total Hrs',
       'PIC Flight Time Total Make-Model', ' ', ' .1'],
      dtype='object')

In [5]:
faa_data = faa_data[
                    ['AIDS Report Number','Local Event Date', 'Event City', 'Event State', 
                    'Event Airport', 'Operator', 'Flight Conduct Code', 'Total Fatalities', 'Total Injuries',
                    'PIC Flight Time Total Hrs', 'PIC Flight Time Total Make-Model' ]
]


In [6]:
def extract_incidents_data(path:str)-> pd.DataFrame:
    """
    Extract incident data from a given file path.

    Args:
        path (str): The file path to the data source.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted incident data.
    """



    faa_data = pd.read_csv(path)
    faa_data = faa_data[
                    ['AIDS Report Number','Local Event Date', 'Event City', 'Event State',
                    'Event Airport', 'Operator', 'Flight Conduct Code', 'Total Fatalities', 'Total Injuries',
                    'PIC Flight Time Total Hrs', 'PIC Flight Time Total Make-Model' ]
]

    return faa_data

In [7]:
path = "data/AIDS_REPORTS.csv"
faa_data = extract_incidents_data(path)
faa_data.head()

,AIDS Report Number,Local Event Date,Event City,Event State,Event Airport,Operator,Flight Conduct Code,Total Fatalities,Total Injuries,PIC Flight Time Total Hrs,PIC Flight Time Total Make-Model
0,20240801009689I,01-AUG-24,SAN DIEGO,CA,SAN DIEGO INTL,JAZZ,FOREIGN AIR CARRIER,0,0,NaN,NaN
1,20240801010149I,01-AUG-24,MANCHESTER,NH,MANCHESTER,NaN,NaN,0,0,3200.0,1600.0
2,20240801010459I,01-AUG-24,BROOMFIELD,CO,ROCKY MOUNTAIN METROPOLITAN,NaN,NaN,0,0,NaN,NaN
3,20240801011259I,01-AUG-24,QUANTICO,VA,QUANTICO MCAF /TURNER FIELD,NaN,NaN,0,0,1145.0,NaN
4,20240801011459I,01-AUG-24,FORT LAUDERDALE,FL,FORT LAUDERDALE EXECUTIVE,NaN,NaN,0,0,4188.0,232.0


In [8]:
faa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   AIDS Report Number                595 non-null    object 
 1   Local Event Date                  595 non-null    object 
 2   Event City                        565 non-null    object 
 3   Event State                       591 non-null    object 
 4   Event Airport                     460 non-null    object 
 5   Operator                          63 non-null     object 
 6   Flight Conduct Code               88 non-null     object 
 7   Total Fatalities                  595 non-null    int64  
 8   Total Injuries                    595 non-null    int64  
 9   PIC Flight Time Total Hrs         472 non-null    float64
 10  PIC Flight Time Total Make-Model  292 non-null    float64
dtypes: float64(2), int64(2), object(7)
memory usage: 51.3+ KB


In [9]:
class PostgreSqlClient:
    """
    A client for querying postgresql database.
    """

    def __init__(
        self,
        server_name: str,
        database_name: str,
        username: str,
        password: str,
        port: int = 5432,
    ):
        self.host_name = server_name
        self.database_name = database_name
        self.username = username
        self.password = password
        self.port = port

        connection_url = URL.create(
            drivername="postgresql+pg8000",
            username="postgres",
            password="postgres",
            host="db-test.cjeesaow22lr.eu-central-1.rds.amazonaws.com",
            port=5432,
            database="postgres",
            #query={"sslmode": "require"}
        )


        self.engine = create_engine(connection_url)

    def select_all(self, table: Table) -> list[dict]:
        return [dict(row) for row in self.engine.execute(table.select()).all()]

    def create_table(self, metadata: MetaData) -> None:
        """
        Creates table provided in the metadata object
        """
        metadata.create_all(self.engine)

    def drop_table(self, table_name: str) -> None:
        self.engine.execute(f"drop table if exists {table_name};")

    def insert(self, data: list[dict], table: Table, metadata: MetaData) -> None:
        metadata.create_all(self.engine)
        insert_statement = postgresql.insert(table).values(data)
        self.engine.execute(insert_statement)

    def overwrite(self, data: list[dict], table: Table, metadata: MetaData) -> None:
        self.drop_table(table.name)
        self.insert(data=data, table=table, metadata=metadata)

    def upsert(self, data: list[dict], table: Table, metadata: MetaData) -> None:
        metadata.create_all(self.engine)
        key_columns = [
            pk_column.name for pk_column in table.primary_key.columns.values()
        ]
        insert_statement = postgresql.insert(table).values(data)
        upsert_statement = insert_statement.on_conflict_do_update(
            index_elements=key_columns,
            set_={
                c.key: c for c in insert_statement.excluded if c.key not in key_columns
            },
        )
        self.engine.execute(upsert_statement)

In [10]:
def load(
    df: pd.DataFrame,
    postgresql_client: PostgreSqlClient,
    table: Table,
    metadata: MetaData,
    load_method: str = "overwrite",
) -> None:
    """
    Load dataframe to either a database.

    Args:
        df: dataframe to load
        postgresql_client: postgresql client
        table: sqlalchemy table
        metadata: sqlalchemy metadata
        load_method: supports one of: [insert, upsert, overwrite]
    """
    if load_method == "insert":
        postgresql_client.insert(
            data=df.to_dict(orient="records"), table=table, metadata=metadata
        )
    elif load_method == "upsert":
        postgresql_client.upsert(
            data=df.to_dict(orient="records"), table=table, metadata=metadata
        )
    elif load_method == "overwrite":
        postgresql_client.overwrite(
            data=df.to_dict(orient="records"), table=table, metadata=metadata
        )
    else:
        raise Exception(
            "Please specify a correct load method: [insert, upsert, overwrite]"
        )

In [11]:
if __name__ == "__main__":
    load_dotenv()
    API_KEY = os.environ.get("API_KEY")
    DB_USERNAME = os.environ.get("DB_USERNAME")
    DB_PASSWORD = os.environ.get("DB_PASSWORD")
    SERVER_NAME = os.environ.get("SERVER_NAME")
    DATABASE_NAME = os.environ.get("DATABASE_NAME")
    PORT = os.environ.get("PORT")

    postgresql_client = PostgreSqlClient(
        server_name=SERVER_NAME,
        database_name=DATABASE_NAME,
        username=DB_USERNAME,
        password=DB_PASSWORD,
        port=PORT,
    )

    df = extract_incidents_data(path)
    
    metadata = MetaData()
    table = Table(
        "incidents_report",
        metadata,
        Column("AIDS Report Number", String, primary_key=True),
        Column("Local Event Date", String),
        Column("Event City", String),
        Column("Event State", String),
        Column("Event Airport", String),
        Column("Operator", String),
        Column("Flight Conduct Code", String),
        Column("Total Fatalities", Integer),
        Column("Total Injuries", Integer),
        Column("PIC Flight Time Total Hrs", Float),
        Column("PIC Flight Time Total Make-Model", Float)
    )

    load(
        df=df,
        postgresql_client=postgresql_client,
        table=table,
        metadata=metadata,
        load_method="upsert",
    )

In [6]:
airports_data = pd.read_csv("/Users/mansourdia/Documents/md/de/de_bc/de_bc/projects/project1/flights_tracking/notebooks/experiments/data_extraction/data/airports.csv")

In [7]:
airports_data

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11.0,NaN,US,US-PA,Bensalem,no,NaN,NaN,K00A,00A,https://www.penndot.pa.gov/TravelInPA/airports...,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,NaN,NaN,00AA,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,NaN,NaN,00AK,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,NaN,NaN,00AL,00AL,NaN,NaN,NaN
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80.0,NaN,US,US-AK,King Salmon,no,NaN,NaN,00AN,00AN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82312,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad
82313,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82314,346788,ZZ-0003,small_airport,Fainting Goat Airport,32.110587,-97.356312,690.0,NaN,US,US-TX,Blum,no,NaN,NaN,87TX,87TX,NaN,NaN,NaN
82315,342102,ZZZW,closed,Scandium City Heliport,69.355287,-138.939310,4.0,NaN,CA,CA-YT,(Old) Scandium City,no,NaN,NaN,NaN,NaN,NaN,NaN,"ZZZW, ZZZW, ZYW, YK96"


In [8]:
airports_data.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'iso_country', 'iso_region',
       'municipality', 'scheduled_service', 'icao_code', 'iata_code',
       'gps_code', 'local_code', 'home_link', 'wikipedia_link', 'keywords'],
      dtype='object')

In [ ]:
def extract_airports_data(path:str)-> pd.DataFrame:
    """
    Extract airports data from a given file path.

    Args:
        path (str): The file path to the data source.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted incident data.
    """



    airports_data = pd.read_csv(path)
    airports_data = airports_data[
                    ['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
                        'elevation_ft', 'continent', 'iso_country', 'iso_region',
                        'municipality', 'scheduled_service', 'icao_code', 'iata_code',
                        'gps_code', 'local_code']
]

    return airports_data